In [26]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")  # Adjust the path to the root directory of your project

import pandas as pd

from Backtesting.BacktestingEngine import *
from StratNaiveLSMom import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Introduction and the Set-up
This notebook is an example on how to use my backtest and data analytic tools.

I choose a sub-universe of my regular trading universe, which consists of 13 coins that are all in the top 30 market cap list (at least at the time of writing this notebook). They all have valid 4-hour OHLCV candlestick data (perpetual future) from Binance from 2022-01-01 to 2024-12-31.
We pre-load the market data and store them in the MarketDataLoader folder.

Our example strategy would use a feature called *momentum score* derived from past returns of each coin.

In [3]:
# Our crypto universe of 13 coins
coins = [
    "BTCUSDT", "ETHUSDT", "BNBUSDT", "XRPUSDT", "ADAUSDT", "DOGEUSDT", "SOLUSDT", 
    "DOTUSDT", "TRXUSDT", "LTCUSDT", "AVAXUSDT", "LINKUSDT", "UNIUSDT", 
]

In [9]:
# Download market data from Binance
marketDataDict = {}
cutOffDate = "2024-12-31 00:00:00"
momScores = pd.read_csv("../MarketDataLoader/mom_scores.csv")
for coin in coins:
    path = f"../MarketDataLoader/OfflineData{coin}_4h_2024-12-31 00:00:00.csv"
    marketDataDict[ coin ] = pd.read_csv( path )                         
    marketDataDict[ coin ]["Open Time"] = pd.to_datetime( marketDataDict[ coin ]["Open Time"] )
    marketDataDict[ coin ].set_index( "Open Time", inplace = True )
    marketDataDict[ coin ] = marketDataDict[ coin ][ marketDataDict[ coin ].index < cutOffDate ]
    marketDataDict[ coin ][ "mom_score" ] = momScores[ coin ].values

In [10]:
marketDataDict[ "BTCUSDT" ].tail()

,Open,High,Low,Close,Volume,Number of Trades,Taker Buy Base,Taker_Avg_Cost,fundingRate,mom_score
Open Time,,,,,,,,,,
2024-12-30 04:00:00,93680.8,94048.9,93181.8,93688.9,16272.993,384330.0,8644.173,93618.360,-4.200000e-07,-0.040559
2024-12-30 08:00:00,93688.9,94053.5,93433.0,93900.1,14170.338,323903.0,7153.472,93785.620,8.791000e-05,-0.038506
2024-12-30 12:00:00,93900.1,94249.9,91510.0,92081.5,70350.030,1266021.0,32753.574,92578.300,8.791000e-05,-0.037643
2024-12-30 16:00:00,92081.5,94668.5,91758.3,94400.0,56405.035,1088662.0,29943.344,93508.580,1.000000e-04,-0.036998
2024-12-30 20:00:00,94400.1,95031.6,91850.0,92759.3,45825.535,844019.0,22837.918,93200.164,1.000000e-04,-0.033937


## A simple Long-Short momentum Strategy

We create an equal weight, long-short momentum strategy based on this `mom_score` feature. Concretely, we long the 6 coins that have the highest `mom_score` in our universe and simultaneously short the 6 coins that have the lowest `mom_score`. The reblance is done at every new candle, i.e., every 4 hours.

We set our backtest starting date as 0+ UTC of 2022-04-01, and our initial capital as 1 million dollars and leverage as 2.

In [16]:
# Create and run the momentum strategy
strategy = MomentumLongShortStrategy( marketDataDict, 
                                      n_top = 6,
                                      n_bottom = 6,
                                      initial_cash = 1e6, 
                                      leverage = 2 )

# Run from 2022-07-01 to 2023-12-31 (after momentum calculation has enough data)
start_time = pd.Timestamp( "2022-04-01" )
end_time = pd.Timestamp( "2024-12-31" )
results = strategy.runStrategy( start_time, end_time )

User defined initialization method recognized: initiating strategy.


  0%|          | 0/6030 [00:00<?, ?it/s]

2022-04-01 12:00:00: LINKUSDT trade closed with PnL: 8468.052347959954
2022-04-01 16:00:00: ETHUSDT trade closed with PnL: 1795.8235675766
2022-04-01 20:00:00: LINKUSDT trade closed with PnL: 1185.1219322925103
2022-04-02 20:00:00: DOTUSDT trade closed with PnL: -13294.779270021832
2022-04-02 20:00:00: TRXUSDT trade closed with PnL: 800.402488108313
2022-04-03 08:00:00: LINKUSDT trade closed with PnL: -3566.2930282081493
2022-04-03 16:00:00: ADAUSDT trade closed with PnL: 8384.530911465288
2022-04-04 00:00:00: AVAXUSDT trade closed with PnL: 7697.311361665231
2022-04-04 16:00:00: DOTUSDT trade closed with PnL: -5665.251865931163
2022-04-04 20:00:00: DOGEUSDT trade closed with PnL: -16785.237883503767
2022-04-04 20:00:00: AVAXUSDT trade closed with PnL: 3570.9501905771526
2022-04-05 00:00:00: DOTUSDT trade closed with PnL: 455.24455023163966
2022-04-05 04:00:00: LINKUSDT trade closed with PnL: -5194.858820735237
2022-04-05 08:00:00: UNIUSDT trade closed with PnL: 8244.14665587434
2022-0

## Performance Analysis Tools


In [21]:
# Analyze and plot results
strategy.plotEquityCurve(results)


In [14]:
# Test my perf stats tools
strategy._performance_cache = None
results_d = strategy._getPerformanceMetrics( results )


In [20]:
# Calculate performance metrics
print(f"Sharpe Ratio: {strategy.computeSharpeRatio(results):.4f}")
print(f"Max Drawdown: {strategy.computeMaxDrawdown(results) * 100:.2f}%")
print(f"Total Return: {strategy.computeTotalReturn(results) * 100:.2f}%")
print(f"CVaR (5%): {strategy.computeCVaR(results, 0.05):.4f}")

# Calculate per-instrument returns
imnt_returns = strategy.computeInstrumentTotalReturn(results)
print("\nPer-Instrument Returns:")
for imnt, ret in imnt_returns.items():
    print(f"{imnt}: ${ret:.2f}")

Sharpe Ratio: 1.3341
Max Drawdown: -40.78%
Total Return: 301.74%
CVaR (5%): -0.0526

Per-Instrument Returns:
BTCUSDT: $306612.43
ETHUSDT: $48107.92
BNBUSDT: $213858.47
XRPUSDT: $917993.29
ADAUSDT: $916510.93
DOGEUSDT: $723264.07
SOLUSDT: $801072.56
DOTUSDT: $588222.46
TRXUSDT: $-889216.48
LTCUSDT: $-784627.61
AVAXUSDT: $1026927.73
LINKUSDT: $-493446.72
UNIUSDT: $116301.91


Without any portforlio optimization or regime filters, this simple strategy already generated a decent Sharpe ratio of 1.33 over the trading horizon. Note that the volatility and returns are skewed by more volatile coins, as this is a equal-weight portfolio.

Not very surprisingly, this strategy suffers a huge drawdown during the 2023 choppy market (It'll be an interesting exercise to think about why). 

In [23]:
strategy.plotDailyPnL( results )

The coin that we lost the most money on is TRX, 
We can drilldown to see the trade history on TRX to which trade caused the huge loss.

In [25]:
trade_history = strategy.getTradesHistoryDf( results )
trade_history[ "TRXUSDT" ].sort_values( "Closed PnL", ascending = True ).head( 5)

,Open Time,Close Time,Entry Price,Direction,Closed PnL,Open PnL
159,2024-12-02 00:00:00,2024-12-04 00:00:00,0.20871,Direction.SHORT,-720935.236006,0.0
155,2024-11-10 12:00:00,2024-11-16 00:00:00,0.16578,Direction.SHORT,-68239.405792,0.0
67,2023-01-06 08:00:00,2023-02-06 04:00:00,0.05048,Direction.SHORT,-46779.883748,0.0
68,2023-02-07 12:00:00,2023-03-10 08:00:00,0.06473,Direction.LONG,-30699.988037,0.0
49,2022-11-09 04:00:00,2022-11-12 04:00:00,0.06276,Direction.LONG,-23908.314916,0.0


The biggest loss (720k Wow!) on TRX came from a two-day short trade entered at 2024-12-02 00:00:00. A quick check on the chart of TRX explains the reason on this huge loss; TRX went from 0.22135 to 0.43347 on 2024-12-03! A sharp 95.82% jump on a single day!

This is the type of event you have to deal with in the crypto world every day! And the way to avoid this kind of huge loss is to implement some risk management measures to control the idiosyncratic risk; either systematically setting stop-loss or get a bot/human to monitor future and current events on coins.
